In [ ]:
#github 仓库名字：Autopilot_V2_Pytorch

# 安装库文件
import h5py
import cv2
import numpy as np

#import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torch.utils.data as Data
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import datetime
import random


# input mudule file
import Py_autopilot2_module

from PIL import Image
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import imageio
import scipy.io as sio 
import pandas as pd
from torchvision.io import read_image


from sklearn.utils import shuffle   
from sklearn.model_selection import train_test_split    #for split
import math    #for pi

In [ ]:
#安装库文件
pip install opencv_python -i https://pypi.mirrors.ustc.edu.cn/simple/       #中科大镜像源
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple opencv_python        #清华源



In [ ]:
#此部分为数据预处理部分

##############################
#数据类型转换
##############################

#数据标准化：
(x-mean(x))/std(x)   ----Z-score

#数据归一化：
(x-min(x))/(max(x)-min(x))


############################
#如何查找本文件下有多少个jpg
############################
ls -lR | grep '.jpg' |wc -l

################################################
#transform 函数调用
################################################
transform_train = transforms.Compose([            #尽量输入PIL image
    # transforms.Resize([h, w]),
    # transforms.RandomCrop(32, padding=4),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #输入数据必须是unint8！可以是pil或者数组。将读取到的图片的维度进行转换（W,H,C转换为C,W,H）并将每个像素值除以255,转化成0-1之间。
    # transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))   #x=(x-mean)/std， 转化成正态分布
    transforms.Normalize((0.4343687,0.492617,0.51987326), (0.22204618,0.24065882,0.29102236))   #x=(x-mean)/std，把对应数据集算mean和std。转化成正态分布
])

# 举例
features = np.array(features).astype(np.uint8)   #把features的数据类型转化成unit8，作为tranform.ToTensor的输入

################################################
#把数据从data.txt中读取出来，写入文件feature和labels.
################################################
def return_data():  

    X = []
    y = []
    features = []

    with open(TRAIN_FILE) as fp:
        for line in islice(fp, LIMIT):
            # print(line)                           #0.jpg ~ 45405.jpg     data.txt Format: 45405.jpg 0.000000
            path, angle = line.strip().split()
            full_path = os.path.join(DATA_FOLDER, path)
            X.append(full_path)
            # using angles from -pi to pi to avoid rescaling the atan in the network
            y.append(float(angle) * scipy.pi / 180)                        #得到pi值，而不是角度值

    # print("lenth=",len(X))  #data.txt lenth= 45406, but actually 45568 pics

    for i in range(len(X)):
        img = plt.imread(X[i])              # X[i]= driving_dataset/0.jpg img shape= (256, 455, 3)
        features.append(preprocess(img))    ## img shape= (100, 100, 3)

    features = np.array(features).astype('float32')
    labels = np.array(y).astype('float32')

    print(features.shape)       #(45406, 100, 100, 3)
    print(labels.shape)         #(45406,)
    with open("features_HSV", "wb") as f:                         #save data
        pickle.dump(features, f, protocol=4)
    with open("labels", "wb") as f:
        pickle.dump(labels, f, protocol=4)

return_data()

########################
#找出label中steer的最大和最小值
#########################

LIMIT = None

DATA_FOLDER = 'driving_dataset'
TRAIN_FILE = os.path.join(DATA_FOLDER, 'data.txt')

def preprocess(img):
    # resized = cv2.resize((cv2.cvtColor(img, cv2.COLOR_RGB2HSV))[:, :, 1], (100, 100))  #取第二个通道
    resized = cv2.resize(img, (100, 100))  #取RGB 3个通道 使用cv2.resize时，参数输入时宽×高×通道，需要注意
    return resized

X = []
y = []
features = []

with open(TRAIN_FILE) as fp:
    for line in islice(fp, LIMIT):
        path, angle = line.strip().split()
        y.append(float(angle))

labels = np.array(y).astype('float32')

# print(features.shape)       #(45406, 100, 100, 3)
print(np.max(labels))         #(45406,)
print(np.min(labels))

##############################################
#计算训练集和测试集的 mean和std（每次新项目新数据集需要重新计算）
##############################################

import cv2
import numpy as np
import torch 
 
#load data shuffle and spilit
features, labels = Py_autopilot2_module.loadFromPickle()   #shape:(45406, 100, 100) (45406,)

#3/7分，并固定分配的随机数random_state=0,    train：31784 + test：13622
train_features, test_features, train_label, test_label = train_test_split(features, labels, test_size=0.3,random_state=0)

#train_features.shape = (31784, 100, 100, 3)
temp_tr = train_features/255 #转化成0-1之间
temp_te = test_features/255 #转化成0-1之间

train_mean_0 = np.mean(temp_tr[:, :, :, 0])   #train dataset 0 channel的mean
train_mean_1 = np.mean(temp_tr[:, :, :, 1])   #train dataset 1 channel的mean
train_mean_2 = np.mean(temp_tr[:, :, :, 2])   #train dataset 2 channel的mean

train_std_0 = np.std(temp_tr[:, :, :, 0])   #train dataset 0 channel的std
train_std_1 = np.std(temp_tr[:, :, :, 1])   #train dataset 0 channel的std
train_std_2 = np.std(temp_tr[:, :, :, 2])   #train dataset 0 channel的std

test_mean_0 = np.mean(temp_te[:, :, :, 0])   #test dataset 0 channel的mean
test_mean_1 = np.mean(temp_te[:, :, :, 1])   #test dataset 1 channel的mean
test_mean_2 = np.mean(temp_te[:, :, :, 2])   #test dataset 2 channel的mean

test_std_0 = np.std(temp_te[:, :, :, 0])   #test dataset 0 channel的std
test_std_1 = np.std(temp_te[:, :, :, 1])   #test dataset 0 channel的std
test_std_2 = np.std(temp_te[:, :, :, 2])   #test dataset 0 channel的std


print(train_mean_0,train_mean_1,train_mean_2)     #0.4343687 0.492617 0.51987326
print(train_std_0,train_std_1,train_std_2)        #0.22204618 0.24065882 0.29102236

print(test_mean_0,test_mean_1,test_mean_2)     #0.43567735 0.49298695 0.5192303
print(test_std_0,test_std_1,test_std_2)        #0.22272868 0.24110594 0.29045662

###################################################################
#存储的图片数量没有变，仅仅dataset和label进行水影评左右翻转，对应数组的长度增加一倍，
###################################################################
#训练数据集左右翻转增加一倍数量，label增加一倍并取相反数， 原shape:(31784, 100, 100, 3) 
train_features_lr = train_features[:,:,::-1,:] #所有照片水平左右翻转
train_features = c = np.concatenate((train_features,train_features_lr),axis=0)    #针对0轴进行拼接 变成(63568, 100, 100, 3) 
train_label_lr = np.negative(train_label)
train_label = np.concatenate((train_label,train_label_lr),axis=0)          #针对0轴进行拼接 变成(63568,) 

In [ ]:
#文件，图片的读写

##################
#读取二进制类型的文件
##################
def loadFromPickle():
    with open("features_RGB", "rb") as f:
        features = np.array(pickle.load(f))
    with open("labels", "rb") as f:
        labels = np.array(pickle.load(f))
    return features, labels





################################################
#把数据从data.txt中读取出来，写入文件feature和labels.
################################################
def return_data():  

    X = []
    y = []
    features = []

    with open(TRAIN_FILE) as fp:
        for line in islice(fp, LIMIT):
            # print(line)                           #0.jpg ~ 45405.jpg     data.txt Format: 45405.jpg 0.000000
            path, angle = line.strip().split()
            full_path = os.path.join(DATA_FOLDER, path)
            X.append(full_path)
            # using angles from -pi to pi to avoid rescaling the atan in the network
            y.append(float(angle) * scipy.pi / 180)                        #得到pi值，而不是角度值

    # print("lenth=",len(X))  #data.txt lenth= 45406, but actually 45568 pics

    for i in range(len(X)):
        img = plt.imread(X[i])              # X[i]= driving_dataset/0.jpg img shape= (256, 455, 3)
        features.append(preprocess(img))    ## img shape= (100, 100, 3)

    features = np.array(features).astype('float32')
    labels = np.array(y).astype('float32')

    print(features.shape)       #(45406, 100, 100, 3)
    print(labels.shape)         #(45406,)
    with open("features_HSV", "wb") as f:                         #save data
        pickle.dump(features, f, protocol=4)
    with open("labels", "wb") as f:
        pickle.dump(labels, f, protocol=4)

#######################
#提取出单个图片进行保存
#######################
img_tmp = data_read['rgb']  #读取h5中保存的200张图片
save_path = "/home/yuxiao/projects/imitation-learning-master/train_data/SeqVal_dataset/" + str(i + h5_cnt*200) + ".png"
imageio.imwrite(save_path,img_tmp[i]) #保存其中一张图片img_tmp[i]为特定名字.png

#######################
#保存.mat文件
#######################
#把字典保存为mat文件
pic_name = str(i + h5_cnt*200) + ".png" #record picture name
label_steer = np.array(input_steer_tmp)
label_steer_sub.update({pic_name: label_steer})
sio.savemat("/home/yuxiao/projects/imitation-learning-master/train_data/SeqVal_mat/label_steer.mat", label_steer_sub)

#######################
#读取.mat文件
#######################
label_dir='/home/yuxiao/projects/imitation-learning-master/train_data/'
label_steer_file='label_steer.mat'
label_steer = sio.loadmat(str(label_dir) + str(label_steer_file))   #dictionary: load all value from steer file
print(label_steer)


############################
#Plt 读取图片并显示单张归一化图片
############################
#读出图片的格式为：
img = plt.imread('driving_dataset/0.jpg')              # X[i]= driving_dataset/0.jpg
img_resize = cv2.resize(img,(100,100))                  #取3个通道,  使用cv2.resize时，参数输入时宽×高×通道，需要注意


img_tp=img   #采取数组操作
pic = (img_tp - np.min(img_tp.flatten()))/(np.max(img_tp.flatten()) - np.min(img_tp.flatten()))#归一化操作图片数据

# pic.swapaxes(0,2)    #交换通道对应轴
plt.imshow(pic)
plt.show()



In [ ]:
#图片，数组的相互转化

##############################################
######### for CV && PIL 相互之间的转化##########
##############################################
#PIL 的数据输入格式： N C W H
#PLT 的数据输入格式： N W H C
#CV2 的数据输入格式： N W H C

# 通过cv2加载的图片，默认色道格式为BGR，可通过cv2.cvtColor函数进行转换；
# 通过PIL加载的图片，默认色道格式为RGB，可通过图像的convert方法进行转换。
def load_img_by_cv(file):
    return cv2.imread(file)     #读出数组array？

def load_img_by_PIL(file):
    return Image.open(file)     #读出pil？

def cv2PIL(img_cv):             #输入array
    return Image.fromarray(cv2.cvtColor(img_cv,cv2.COLOR_BGR2RGB))

def PIL2cv(img_pil):            #输入PIL
    return cv2.cvtColor(np.asarray(img_pil),cv2.COLOR_RGB2BGR)

    
#################################
#array 转成PIL img
#################################
from PIL import Image
img_array = np.array(train_features[0,:,:,:]).astype(np.uint8)
img_temp = Image.fromarray(img_array)  #array 转成img
# transform_train(img_temp)
print(img_temp[0])

#第二种方式
img_arr = torch.from_numpy( np.array(train_features[0]) )
answer = transforms.ToPILImage(mode='RGB')(img_arr)
answer[:,:,0].show()

#################################
#PIL img 转成 array
#################################
X = []
y = []
features = []

with open(TRAIN_FILE) as fp:
    for line in islice(fp, LIMIT):
        # print(line)                           #0.jpg ~ 45405.jpg     "data.txt" data Format: 45405.jpg 0.000000
        path, angle = line.strip().split()
        full_path = os.path.join(DATA_FOLDER, path)
        X.append(full_path)


for i in range(len(X)):
    img = plt.imread(X[i])              # X[i]= driving_dataset/0.jpg; img shape= (256, 455, 3)
    features.append(preprocess(img))    ## img shape= (100, 100, 3)

features = np.array(features).astype('float32')   #使用cv2的时候要注意swap

#################################
#RGB 转成 HSV
#################################

# Opencv提供了cvtColor函数，调用该函数可以非常方便地实现不同颜色空间的转换。不过为了可视化，调用该函数得到的HSV图像，
# 其H、S、V三通道的取值范围并不是0~360、0~1、0~1，而是经过转换的0~180、0~255、0~255。

# https://www.cnblogs.com/weijiakai/p/15495607.html

In [ ]:
#显示照片

########################
#直接显示一张图片（输入为数组）
########################
plt.imshow(img_get[0].cpu().numpy().T)
plt.show()

#######################
#显示归一化的多张图片（plt方式）
#######################
import h5py
import numpy as np

print(features.shape)
# print(features[0])

# show iamge to see!
# print first img in BATCH , check the input data should be   N x W x H x C

img_tp = np.array(features[0,:,:,:])
img_tp.swapaxes(0,2)
temp = (img_tp - np.min(img_tp.flatten()))/(np.max(img_tp.flatten()) - np.min(img_tp.flatten()))
plt.imshow(temp)
plt.show()


###################
#显示数组对应（PIL方式）
###################
fea_org = features[0:4]  #显示4张照片
print(fea_org.shape)

from PIL import Image
import numpy as np
# data is your array

img = Image.fromarray(fea_org[0], 'RGB')
img.show()
img = Image.fromarray(fea_org[1], 'RGB')
img = Image.fromarray(fea_org[2], 'RGB')
img = Image.fromarray(fea_org[3], 'RGB')
img.show()

##########################
#动态显示图片（运用流媒体的方式）
##########################
import os
import cv2
import ipywidgets as widgets
from IPython.display import display

imgbox = widgets.Image(format='jpg', height=300, width=400)
display(imgbox)
root = "/media/yuxiao/C1/projects/Autopilot-master/Autopilot_V2/driving_dataset"
for file in os.listdir(root):
    canvas = cv2.imread(os.path.join(root, file))
    imgbox.value = cv2.imencode('.jpg', canvas)[1].tobytes()

# 原文链接：https://blog.csdn.net/liuqixuan1994/article/details/88715454

In [ ]:
# 数组&矩阵的常规操作

##########
#数组拼接
##########
a=np.array([[1,2,3],[4,5,6]])
b=np.array([[21,22,23],[7,8,9]])
c=np.concatenate((a,b),axis=0)
print(a.shape,b.shape,c.shape)
print(a)
print(c)

#########
#


##########################
#把m x n整个矩阵展平
##########################
img_tp = img_get[0,:,:,:].numpy()
img_tp.flatten()




In [ ]:
#字典的处理

##########################
#显示字典的值 .keys  .values
##########################
#把字典的key和value的值取出来,按照顺序存入到list中 
#创建字段
d={'name':'cheng','age':20,'sex':'female'}
#创建空列表
a=[]
#将字典中键和值循环取出添加到列表中
for i in d.keys():
a.append(i)
a.append(d[i])
print a

##########################
#求字典最大最小值
##########################
x=label_steer.values()  #取出所有values为字典
y=list(x)               #字典转化为list
y=np.array(y).squeeze() #list转化为np，（）增加维度才能变为数组
z=np.empty(3200,float)
for i in range(3200):
    z[i]= y[i][0]
print(min(z),max(z))
或者
min(zip(d.values(), d.keys()))
或者
x=label_steer.values()  #取出所有values为字典
y=list(x)               #字典转化为list
y=np.array(y).squeeze() #list转化为np，并减少维度
y = x[:,0]              #取每一组的[0]

##########################
#计算字典alues的zscore值
##########################
import scipy.stats as stats
keys, vals = zip(*d.items())
z = stats.zscore(vals)

##########################
#合并两个list变成一个字典
##########################
newmap = dict(zip(keys,z))

##########################
#去掉字典里的值  （pop）
##########################
label_steer.pop('__header__')

In [ ]:
#数据类型转换

##########################
# CPU tensor和GPU tensor之间的转换
##########################
# （1）从CPU tensor到GPU tensor，使用
data.cuda()

# （2）从GPU tensor到CPU tensor，使用
data.cpu()

##########################
# Tensor与Numpy array之间的转换
##########################

# （1）Tensor到Numpy array可以使用
data.numpy()        #其中data的类型为torch.Tensor。

# （2）Numpy array到Tensor可以使用
torch.from_numpy(data)      #其中data的类型为numpy.ndarray。

##########################
# CPU tensor之间的转换或GPU tensor之间的转换
##########################

# （1）一般只要在tensor后加long()，int()，double()，float()，byte()等函数就能将tensor进行类型转换。

# 例如：Torch.LongTensor转换为Torch.FloatTensor，直接使用data.float()即可。

# （2）还可以使用type()函数。

# 当data为tensor数据类型，如果使用data.type(torch.FloatTensor)则强制转换data为torch.FloatTensor类型张量。

# （3）当不知道要转换为什么数据类型，但需要求a1，a2两个张量的乘积时，可以使用a1.type_as(a2)将a1转换为a2同类型。

##########################
# 数组astype的使用
##########################
features = np.array(features).astype('float32')


In [ ]:
#可视化参数

#########################
#visdom动态显示loss 和epoch
#########################

# pip install visdom
# python -m visdom.server
# visdom成功启动后，会返回一个网址，根据显示的网址然后在浏览器里输入：http://localhost:8097 进行登录

from visdom import Visdom
import time

# 将窗口类实例化
viz = Visdom() 

# 创建窗口并初始化
viz.line([[0.,0.]], [0], win='train', opts=dict(title='loss&acc', legend=['loss', 'acc']))
for global_steps in range(10):
    # 随机获取loss和acc
    loss = 0.1 * np.random.randn() + 1
    acc = 0.1 * np.random.randn() + 0.5
    # 更新窗口图像
    viz.line([[loss, acc]], [global_steps], win='train', update='append')
    # 延时0.5s
    time.sleep(0.5)

In [ ]:
#保存网络

####################
# 仅仅保存网络当前的参数
####################
model_par_name = 'results/model_img_par_{}.pkl'.format(epoch + 1)  #save each epoch
torch.save(model.state_dict(), model_par_name)

####################
# 保存整个网络
####################
model_name = 'models/model_{}.pkl'.format(epoch + 1)  #save each epoch
torch.save(model, model_name)  #保存网络和参数  

In [ ]:
#debug

#########################
# 查网络输出问题：
#########################
# 先检查输入的数据
# 然后逐步检查每一层网络的输入输出

# 类中运用self.x  进行追踪每一层网络的输出
# self.out = model.out来查看




In [ ]:
#tensorflow 改写成pytorch

#######################################################################
#tensorflow转换pytorch问题padding='same' conversion to PyTorch padding=#
#######################################################################
###############################################
# tensorflow转化成pytorch时候pads问题
# padding='same' conversion to PyTorch padding=#
###############################################

# https://stackoverflow.com/questions/62166719/padding-same-conversion-to-pytorch-padding/62167204


# In convolution padding = 1 for 3x3 kernel and stride=1 is ~ "same" in keras.

# And In MaxPool you should set padding=0 (default), for 2x2 kernel, stride=2 is ~ "same" in keras.

# You can use Formula:

# Out = (W+2P-K)/S + 1

# Let see some mathematical calculation:

# For Convolution:

# Case 1:

# input is 30x30, kernel_size(K) is 3x3, stride=1, padding=1:

# Out = (30+2*1-3)/1 + 1 = floor(29/1) + 1 = 30 i.e 30x30 (~ padding="same")

# Case 2:

# input is 30x30, kernel_size(K) is 3x3, stride=1, padding=0:

# Out = (30+2*0-3)/1 + 1 = floor(27/1) + 1 = 28 i.e 28x28 (~ padding="valid")

# For MaxPooling:

# Case 1:

# input is 30x30, kernel_size(K) is 2x2, stride=2, padding=0:

# Out = (30+2*0-2)/2 + 1 = floor(28/2) + 1 = 15 i.e 15x15 (~ padding="same")

# Case 2:

# input is 30x30, kernel_size(K) is 2x2, stride=2, padding=1:

# Out = (30+2*1-2)/2 + 1 = floor(30/2) + 1 = 16 i.e 16x16 (~ padding="valid")



In [ ]:
#安装carla遇到的问题总结
###################################################################################
# ubuntu20.04 系统因为做了系统升级，自动卸载了一些没用的包，到这原来可以运行的carla不能启动报错：
# /opt/carla-simulator/CarlaUE4/Binaries/Linux/CarlaUE4-Linux-Shipping: error while loading shared libraries: 
# libomp.so.5: cannot open shared object file: No such file or directory

# 解决方法：
# 尝试重新安装 libosm5 包 
# sudo apt-get install libomp5
# 安装完成后再次启动carla成功。

############################################
# ERROR： GlobalShaderCache-GLSL_150_ES31.bin

# 解决办法：
# 安装游戏引擎vulkan
# sudo apt install vulkan-utils